# Calculate a 2D emissions grid incorporating COVID-19 effects for short-lived forcers

In [1]:
from datetime import datetime
import netCDF4 as nc
import pandas as pd
import numpy as np
import reverse_geocoder as rg
import matplotlib.pyplot as plt

import os
import sys

sys.path.append(os.getcwd())
from utils import copy_netcdf_file, insert_interpolated_point, cutoff_netcdf_time

In [2]:
input_folder = "../input/aerosols/"
output_folder = "../output/aerosols/daily/"

# Input for the aerosols/gases
input_nox = "NOx-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_bc = "BC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_so2 = "SO2-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_oc = "OC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_co = "CO-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_nh3 = "NH3-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
input_nmvoc = "NMVOC-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"

convert_country_code_file = "convertCountryCodes.csv"
# Input blip is the emissions change, found in the github folder
git_folder = "../../COVID19_emissions_data/"
input_blip = "sector_emissions/Covid_CO2emissions_sectors_May30.csv"
files_to_blip = [
    input_co, 
    input_nh3, input_nmvoc,
    input_oc, input_so2, input_nox, input_bc
]
key_variables = [
    "CO_em_anthro", 
    "NH3_em_anthro", "NMVOC_em_anthro",
    "OC_em_anthro", "SO2_em_anthro", "NOx_em_anthro", "BC_em_anthro"
]
# The name to affix on the variables output
scenario_string = "daily_v1.nc"

def convert_years_to_days(year):
    return (year - 2015) * 365

def convert_2020_days_to_netcdf_days(days):
    return days + 5 * 365  # Our netcdf files start in 2015 rather than 2020. 

# We only want data from 2020. Data is recorded in 365-day years, so we correct the time of ending. 
tstart = convert_years_to_days(2020)
tcutoff = convert_years_to_days(2021 + 0.000001) # We want to include the leapday, the small number ensures inclusion
tcutoff_initial = convert_years_to_days(2030 + 0.6 / 12) # We want to include the first month after 2030

# We apply a uniform rate to discount some time period between flatrate_start and flatrate_end
flatrate_start =  convert_years_to_days(2020.6)
flatrate_end = tcutoff

In [3]:
# More inputs for the long-term path modification
# Modification factors found at:
mod_input_folder = git_folder + "global_pathways/"
mod_input_baseline = "Base_pathway.xlsx"
mod_input_ff = "FossilFuel_pathway.xlsx"
mod_input_2yr= "TwoYearBlip_pathway.xlsx"
mod_input_mg = "ModerateGreen_pathway.xlsx"
mod_input_sg = "StrongGreen_pathway.xlsx"

# Map between key variable name and excel column
path_var_names = {
    "SO2_em_anthro": "sox(MtS/year)",  
    "CO_em_anthro": "co(Mt/year)", 
    "NMVOC_em_anthro": "nmvoc(Mt/year)",
    "NOx_em_anthro": "nox(MtN/year)",
    "BC_em_anthro": "bc(Mt/year)",
    "OC_em_anthro":"oc(Mt/year)", 
    "NH3_em_anthro": "nh3(Mt/year)"
}

pathway_files = [mod_input_baseline, mod_input_ff, mod_input_2yr, mod_input_mg, mod_input_sg]

modify_range = np.arange(2022, 2051.01)

cut_str = "cut_"
baseline_string = "_baseline.nc"

In [4]:
assert len(files_to_blip) == len(key_variables) # check input

## Collect and clean the data
We need to make the blip factors consistent with the netCDF data structure. This will require example data, although the results should not depend which example is chosen. 

In [5]:
nox_0 = nc.Dataset(input_folder + input_nox, "r", format="NETCDF4")
blip_factors = pd.read_csv(git_folder + input_blip)
convert_countries = pd.read_csv(input_folder + convert_country_code_file, keep_default_na=False, na_values=['_'])

In [6]:
data_to_modify = [nox_0]

In [7]:
for dimobj in nox_0.dimensions.values():
...     print(dimobj)

<class 'netCDF4._netCDF4.Dimension'>: name = 'lon', size = 720
<class 'netCDF4._netCDF4.Dimension'>: name = 'lat', size = 360
<class 'netCDF4._netCDF4.Dimension'>: name = 'sector', size = 8
<class 'netCDF4._netCDF4.Dimension'> (unlimited): name = 'time', size = 120
<class 'netCDF4._netCDF4.Dimension'>: name = 'bound', size = 2


In [8]:
nox_0.variables["sector"]

<class 'netCDF4._netCDF4.Variable'>
int32 sector(sector)
    long_name: sector
    bounds: sector_bnds
    ids: 0: Agriculture; 1: Energy; 2: Industrial; 3: Transportation; 4: Residential, Commercial, Other; 5: Solvents production and application; 6: Waste; 7: International Shipping
unlimited dimensions: 
current shape = (8,)
filling on, default _FillValue of -2147483647 used

In [9]:
blip_factors = blip_factors[~blip_factors["1"].isna()]
blip_sectors = blip_factors["Sector"].unique()

## Perform the sector weighting


In [10]:
blip_sectors

array(['total', 'surface-transport', 'residential', 'public/commercial',
       'industry', 'international-shipping', 'international-aviation',
       'domestic-aviation', 'power'], dtype=object)

The set of sectors in our blip need to be converted into our sectors in the netCDF case. This uses:
0: Agriculture; 1: Energy; 2: Industrial; 3: Transportation; 4: Residential, Commercial, Other; 5: Solvents production and application; 6: Waste; 7: International Shipping

In [11]:
sector_dict = {
    "surface-transport": 3, "residential": 4, "public/commercial": -4, "industry": 2, 
    "international-shipping":7, "international-aviation": -1, 
    "domestic-aviation": -2, "power": 1, "total": -5
}
# We will manage aviation elsewhere, no change to sector 0 (agri) and no need of the total.
sectors_to_use = [1, 2, 3, 4, 5, 7]  

In [12]:
blip_factors_multi = blip_factors.copy()
blip_factors_multi.drop(["Country", "Base(MtCO2/day)", "Unnamed: 0"], axis=1, inplace=True)
blip_factors_multi["Sector"] = [sector_dict[sect] for sect in blip_factors_multi["Sector"]]
blip_factors_multi.head()

,ISO_A3,Sector,Base%,1,2,3,4,5,6,7,...,357,358,359,360,361,362,363,364,365,366
0,ALB,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,DZA,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346
2,AGO,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926
3,ARG,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269
4,ARM,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [13]:
blip_factors_multi.set_index(blip_factors_multi.columns[:2].to_list(), drop=True, inplace=True)
blip_factors_multi.head()

,,Base%,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
ISO_A3,Sector,,,,,,,,,,,,,,,,,,,,,
ALB,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DZA,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346
AGO,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926
ARG,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269
ARM,-5,100.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [14]:
# We want to average the two sets of sector 4 together in the right ratio
all_countries = blip_factors_multi.index.get_level_values("ISO_A3").unique()
for country in all_countries:
    if (country, 4) in blip_factors_multi.index and (country, -4) in blip_factors_multi.index:
        blip_factors_multi.loc[country, 4] = (
            blip_factors_multi.loc[country, 4].values * 
            blip_factors_multi["Base%"][country, 4] + 
            blip_factors_multi.loc[country, -4].values * blip_factors_multi["Base%"][country, -4]
            ) / (
                blip_factors_multi["Base%"][country, 4] + blip_factors_multi["Base%"][country, -4]
            )
        blip_factors_multi["Base%"][country, 4] = blip_factors_multi["Base%"][country, 4] + \
            blip_factors_multi["Base%"][country, -4]
        blip_factors_multi.drop((country, -4), inplace=True)
    elif (country, -4) in blip_factors_multi.index:
        blip_factors_multi.loc[country, 4] = blip_factors_multi.loc[country, -4]
    elif (country, 4) in blip_factors_multi.index:
        continue
    else:
        print("no data for {}".format(country))

In [15]:
# Test that this produces the right answers
example_factor = blip_factors[
    (blip_factors["ISO_A3"] == "GBR") & (blip_factors["Sector"].isin(["residential", "public/commercial"]))
][["Base%", "100"]]
assert np.isclose(blip_factors_multi.loc["GBR", 4][100], sum(
    example_factor["Base%"] * example_factor["100"]) / sum(example_factor["Base%"])
)

In [16]:
blip_factors_multi

,,Base%,1,2,3,4,5,6,7,8,9,...,357,358,359,360,361,362,363,364,365,366
ISO_A3,Sector,,,,,,,,,,,,,,,,,,,,,
ALB,-5,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
DZA,-5,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346,-0.171346
AGO,-5,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926,-0.126926
ARG,-5,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269,-0.197269
ARM,-5,100.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
VEN,1,43.093385,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915,-0.096915
VNM,1,31.822661,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.035497,0.035497,0.035497,0.035497,0.035497,0.035497,0.035497,0.035497,0.035497,0.035497
YEM,1,34.652509,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276,-0.023276


We additionally assume that solvent production tracks industry.

In [17]:
for country in all_countries:
    blip_factors_multi.loc[(country, 5), :] = blip_factors_multi.loc[(country, 4), :]

KeyboardInterrupt: 

In [ ]:
blip_factors_multi.index.levels

## Derive country and date relation
We need to assign each lat/long a country. This is slightly complicated by the country index being 2 letters in the inverse geocoder but 3 letters in our data.

In [ ]:
lat, lon = nox_0.variables["lat"][:], nox_0.variables["lon"][:]

In [ ]:
convert_countries_dict = {convert_countries["A2 (ISO)"][i]: convert_countries["A3 (UN)"][i] for i in convert_countries.index}
coords = []
lon_length = len(lon)
for latperm in lat:
    coords = coords + list(zip([latperm] * lon_length, lon))

In [ ]:
results = rg.search(coords)

In [ ]:
lat_countries_dict = {coords[i]: convert_countries_dict[results[i]["cc"]] for i in range(len(coords)) 
                      if results[i]["cc"] in convert_countries_dict.keys()}

The process will be faster if we map the other way and use the index rather than the coordinates:

In [ ]:
country_coord_dict = {}
for k, v in lat_countries_dict.items():
    country_coord_dict[v] = country_coord_dict.get(v, [])
    country_coord_dict[v].append((np.where(lat.data == k[0])[0][0], np.where(lon.data == k[1])[0][0]))

Now we must relate the dates. blip_factors uses days from 2020-01-01, and has values for every day. The netCDFs use days since 2015-01-01, which is 5 * 365 days later and monthly. (The netcdf does not use leap days)

In [ ]:
netCDF_times = nox_0.variables["time"][:]
netCDF_tseries = pd.Series(netCDF_times)
bliptimes = blip_factors_multi.columns[blip_factors_multi.columns != "Base%"]
bliptimes = pd.Series(pd.to_numeric(bliptimes))

In [ ]:
blip_factors_use = blip_factors_multi.copy()
del blip_factors_use["Base%"]
blip_factors_use.columns = [int(col) for col in blip_factors_use.columns]

## Perform the emissions blip
We now have a mapping between times and locations and the emissions we want. 

In [ ]:
all_valid_countries = [c for c in all_countries if c in country_coord_dict.keys()]
day_range = list(np.arange(tstart, tcutoff))
to_interp_range = [day for day in day_range if day not in nox_0.variables["time"][:]]

In [ ]:
nox_0.close()

In [19]:
varrate_timeinds = [
    ind for ind in bliptimes if (convert_2020_days_to_netcdf_days(ind) < flatrate_start)
]
varrate_ind = max(blip_factors_use.columns)
varrate_ind
crop_str = "_crop.nc"
mod_str = "_modified.nc"
working_str = "_workings"

NameError: name 'bliptimes' is not defined

In [ ]:
for fileind in range(len(files_to_blip)):
    file = files_to_blip[fileind]
    data = cutoff_netcdf_time(input_folder, output_folder, file, tcutoff_initial, scenario_string, compress=False)
    print("Working on data for {}".format(key_variables[fileind]))
    # Insert all the required days
    start = datetime.now()
    for day in to_interp_range:
        insert_interpolated_point(data, day, 1, 1)
    end = datetime.now()
    print("Finished inserting data. Took {}".format(end - start))
    data.close() 
    # We now want to cut off the extraneous times. 
    start = datetime.now()
    data = cutoff_netcdf_time(
        output_folder, output_folder, cut_str + file + scenario_string, tcutoff, crop_str, remove_string=cut_str,
        compress=False, tstart=tstart
    )
    end = datetime.now()
    print("Finished cropping data. Took {}".format(end - start))
    data.close()
    # Now we copy the baseline data
    data = copy_netcdf_file(
        cut_str + file + scenario_string + crop_str, output_folder, output_folder, 
        scenario_string=baseline_string, compress=True, remove_string=crop_str
    )
    data.close()
    print("cropped baseline")
    # Other files will need modification
    data = copy_netcdf_file(
        cut_str + file + scenario_string + crop_str, output_folder, output_folder, 
        scenario_string=working_str, compress=False, remove_string=crop_str
    )
    output = data.variables[key_variables[fileind]][:, :, :, :]
    plt.figure(figsize=(16, 8))
    plt.plot(data.variables["time"][:]/365 + 2015, output[:,  2, 262, 605])
    flatrate_timeinds = np.where(
        (data.variables["time"][:] <= flatrate_end) & (data.variables["time"][:] >= flatrate_start)
    )[0]
    # Modify points by a time-and country dependent value in 2020 - 2021
    for country in all_valid_countries:
        print(country)
        for time in varrate_timeinds:
            timeind = np.where(data.variables["time"][:] == convert_2020_days_to_netcdf_days(time))[0]
            for sector in sectors_to_use:
                try:
                    mult_fact = blip_factors_use[time].loc[country, sector] + 1
                    if mult_fact != 1.0: #This saves operations
                        for lati, longi in country_coord_dict[country]:
                            output[timeind, sector, lati, longi] *= mult_fact
                except KeyError as e:
                    continue
                    
        # Then make the changes for all of the flat rate times (same factor)
        timeinds = flatrate_timeinds
        for sector in sectors_to_use:
            try:
                mult_fact = blip_factors_use[varrate_ind].loc[country, sector] + 1
                if mult_fact != 1.0: #This saves operations
                    for lati, longi in country_coord_dict[country]:
                        output[timeinds, sector, lati, longi] *= mult_fact
            except KeyError as e:
                continue
    plt.plot(data.variables["time"][:]/365 + 2015, output[:,  2, 262, 605], linestyle="--")
    data.variables[key_variables[fileind]][:, :, :, :] = output
    data.close()
    # Finally compress this to make a useable file. 
    data = copy_netcdf_file(
        cut_str + file + scenario_string + working_str, output_folder, output_folder, 
        scenario_string=mod_str, compress=True, remove_string=working_str
    )
    data.close()
    

## SANDBOX
Code below this line will break, but may be useful to use in some order to understand the data. 

In [ ]:
blob  # unassigned variable stops execution

In [ ]:
data.close()

In [ ]:
old_data = nc.Dataset(
    "../input/aerosols/CO-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.nc"
)

In [ ]:
data = nc.Dataset(
    "../output/aerosols/daily/cut_CO-em-anthro_input4MIPs_emissions_ScenarioMIP_IAMC-MESSAGE-GLOBIOM-ssp245-1-1_gn_201501-210012.ncdaily_v1.nc_baseline.nc"
)

In [ ]:
output=data.variables["SO2_em_anthro"][...]

In [ ]:
for fileind in range(len(files_to_blip)):
    file = files_to_blip[fileind]
    data = nc.Dataset(input_folder + file)
    plt.plot(np.arange(12), data.variables[key_variables[fileind]][12:24, 2, 262, 605]/data.variables[key_variables[fileind]][12, 2, 262, 605] )

In [21]:
for fileind in [6]: #range(len(files_to_blip)):
    file = files_to_blip[fileind]
    data = copy_netcdf_file(
        cut_str + file + scenario_string + working_str, output_folder, output_folder, 
        scenario_string=mod_str, compress=True, remove_string=working_str
    )
    data.close()

In [20]:
crop_str = "_crop.nc"
mod_str = "_modified.nc"
working_str = "_workings"

In [ ]:
plt.plot(data.variables["time"][:]/365 + 2015, output[:,  2, 262, 605])
for time in monthly_range:
    timeind = np.where(data.variables["time"][:] == time)
    time_factor = path_df[path_var_names[key_variables[fileind]]].loc[
        make_day_into_year(time)
    ] / baseline[path_var_names[key_variables[fileind]]].loc[make_day_into_year(time)]
    output[timeind, ...] = (output[timeind, :]) * time_factor 
plt.plot(data.variables["time"][:]/365 + 2015, data.variables[key_variables[fileind]][:,  2, 262, 605])


In [ ]:
plt.figure(figsize=(16, 8))
data = nc.Dataset(input_folder + file)
plt.plot(data.variables["time"][:100] / 365, data.variables[key_variables[fileind]][:100, 2, 262, 605])
data = nc.Dataset(output_folder +"cut_" + file + scenario_string + "_" + path_string)
plt.plot(data.variables["time"][:100] / 365, data.variables[key_variables[fileind]][:100, 2, 262, 605], linestyle= "--")
